In [2]:
import pandas as pd
import bw2data as bd
import bw2io as bi
import bw2analyzer as ba
import bw2calc as bc
import bw_processing as bwp
from pathlib import Path
import numpy as np
import csv
import sys

In [3]:
bd.projects.set_current('ei38-teaching-25')
ipcc=('IPCC 2013', 'climate change', 'GWP 100a')

In [28]:
ei = bd.Database('ei 3.8 cutoff')
activity= ei.get('6a7848cd768f5163834eec59c80de49b')
ei_pack=ei.datapackage()

In [5]:
#[act for act in ei if 'market for acrylic acid' in  act['name']][1].id

15498

In [26]:
heat_acts=[act for act in ei if 'heat production' in  act['name']
#and ('RER' in act['location'] or 'DE' in act['location'] or 'Europe without Switzerland' in act['location'])
#and 'heat' in act['reference product'] 
and 'heat pump' not in act['name']
and 'biomethane' not in act['name']
#and ('natural gas' in act['name'] or 'anthracite' in act['name'])
]

In [52]:
exchange =list(activity.technosphere())[0]
a= np.array((exchange.input.id, exchange.output.id), bwp.INDICES_DTYPE)

In [55]:
np.where(ei_pack.data[5] == a)

(array([157541], dtype=int64),)

In [32]:
index=[]
flow_list=[]
global depth
depth=0
max_depth=2

In [9]:
ei_pack.data[5][3]['row']

4428

In [56]:
def recurse_supply(act):
    global depth
    for exc in act.technosphere():
        if exc.input in heat_acts:
            flow_list.append(np.array((exc.input.id, exc.output.id), bwp.INDICES_DTYPE))
        elif depth< max_depth:
            depth +=1 
            recurse_supply(exc.input)
    depth -= 1
        

In [57]:
flow_list=[]
global depth
depth=0
max_depth=3
recurse_supply(activity)

In [58]:
flow_list

[array((17601, 17526), dtype=[('row', '<i4'), ('col', '<i4')]),
 array((22406, 19964), dtype=[('row', '<i4'), ('col', '<i4')]),
 array((5258, 19964), dtype=[('row', '<i4'), ('col', '<i4')]),
 array((15159, 19964), dtype=[('row', '<i4'), ('col', '<i4')]),
 array((22127, 6767), dtype=[('row', '<i4'), ('col', '<i4')]),
 array((20443, 6767), dtype=[('row', '<i4'), ('col', '<i4')]),
 array((22127, 23741), dtype=[('row', '<i4'), ('col', '<i4')]),
 array((20443, 23741), dtype=[('row', '<i4'), ('col', '<i4')]),
 array((16708, 20823), dtype=[('row', '<i4'), ('col', '<i4')]),
 array((18519, 20823), dtype=[('row', '<i4'), ('col', '<i4')]),
 array((6998, 20823), dtype=[('row', '<i4'), ('col', '<i4')]),
 array((18590, 20823), dtype=[('row', '<i4'), ('col', '<i4')]),
 array((23197, 20823), dtype=[('row', '<i4'), ('col', '<i4')]),
 array((4916, 20823), dtype=[('row', '<i4'), ('col', '<i4')]),
 array((15721, 20823), dtype=[('row', '<i4'), ('col', '<i4')]),
 array((16902, 20823), dtype=[('row', '<i4'),